In [121]:
import lxml.html
import lxml.etree
import requests

## Находим коммисии и сохраняем их адреса

In [122]:
#main site
req = requests.get("http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222")
req.status_code

200

In [123]:
tree = lxml.html.fromstring(req.text)

In [124]:
comms = tree.xpath('//a[@style = "text-decoration: none"]')

## И с сайта каждой коммисии извлекаем данные и УИК'ах и записываем в DataFrame

In [125]:
import pandas as pd

df = pd.DataFrame()

In [126]:
import re
pattern = re.compile(r'(.+)%')

In [127]:
#Смотрим коммисию
for com_num in range(30):
    req = requests.get(comms[com_num].attrib['href'])
    req.status_code

    tree = lxml.html.fromstring(req.text)




    #Считываем УИК'и
    wick_line = tree.xpath('/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[1]/td//text()')
    #Собираем данные коммисии
    data = []
    data.append([comms[com_num].text]*len(wick_line))
    data.append(wick_line)
    #Считываем значения

    # с 2 по 12 включительно
    for numbers in range(2,13):
        data_line = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{numbers}]/td//b/text()')
        data.append(data_line)

    # c 14 по 16 данные по участникам
    for guys in range(14,17):
        data_line = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{guys}]/td//b/text()')
        data.append(data_line)
        # проценты
        data_line = tree.xpath(f'/html/body/table[@style = "padding-left: 3px; padding-right: 3px;"]/tr[@height = "100%"]/td/table[@style = "width:100%;border-color:#000000"]/tr/td[2]/div/table/tr[{guys}]/td')
        # получив вектор из td нужно получить процентное значение без тэга, для чего берётся весь текст из td и с помощью регулярных выражений получает проценты
        data_line = list(map(lambda x:pattern.findall(x.text_content())[0],data_line))
        data.append(data_line)


    df = df.append(pd.DataFrame(data).transpose(),ignore_index = True)

## Добавляем названия столбцов 

In [128]:
df.columns =['№ Коммисии','№ УИК','Число избирателей, внесенных в список избирателей на момент окончания голосования'
             ,'Число избирательных бюллетеней, полученных участковой избирательной комиссией','Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования'
             ,'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования','Число погашенных избирательных бюллетеней'
             ,'Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования','Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования'
             ,'Число недействительных избирательных бюллетеней','Число действительных избирательных бюллетеней','Число утраченных избирательных бюллетеней'
             ,'Число избирательных бюллетеней, не учтенных при получении','Амосов Михаил Иванович','% Амосова','Беглов Александр Дмитриевич','% Беглова'
             ,'Тихонова Надежда Геннадьевна','% Тихоновой']
df[:5]

,№ Коммисии,№ УИК,"Число избирателей, внесенных в список избирателей на момент окончания голосования","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",Число погашенных избирательных бюллетеней,"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования","Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Амосов Михаил Иванович,% Амосова,Беглов Александр Дмитриевич,% Беглова,Тихонова Надежда Геннадьевна,% Тихоновой
0,Территориальная избирательная комиссия №1,УИК №1,1803,1200,587,11,602,11,587,23,575,0,0,110,18.39,345,57.69,120,20.07
1,Территориальная избирательная комиссия №1,УИК №2,1466,1100,433,14,653,14,430,19,425,0,0,53,11.94,326,73.42,46,10.36
2,Территориальная избирательная комиссия №1,УИК №3,2092,1600,576,22,1002,22,576,15,583,0,0,155,25.92,332,55.52,96,16.05
3,Территориальная избирательная комиссия №1,УИК №4,1056,1000,318,4,678,4,318,13,309,0,0,67,20.81,171,53.11,71,22.05
4,Территориальная избирательная комиссия №1,УИК №5,1827,1400,495,8,897,8,493,16,485,0,0,137,27.35,266,53.09,82,16.37


## И выводим в CSV файл

In [129]:
csv = df.to_csv('Elections.csv',encoding = "cp1251")